In [ ]:
"""Notebook designed to do differential analysis on LCMS results based on batch, type of sample and various other conditons.  Reads in LCMS results
files from multiple experiements, performs various normalizations for data vizualization and exploration, and matches differential protiens with
protein annotation for further exploration"""

In [ ]:
import pandas as pd
import os
from pandas import Series
import numpy as np
import functools
import argparse
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from skbio import DistanceMatrix
from skbio.tree import nj
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from sklearn.decomposition import PCA

%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [ ]:
"""Reading in data and annotations"""
data_dir = '/Users/home/data/'
genome_annotations = pd.read_csv(data_dir + 'annotations.csv')
metadata = pd.read_excel(data_dir + 'metadata.xlsx', index_col='sample_name')

"""Reading in multiple LCMS files from multiple experiments for larger comparison sets"""
lcms_1 = pd.read_csv(data_dir + 'lcsms_file_1.csv', index_col='Accession') 
lcms_2 = pd.read_csv(data_dir + 'lcsms_file_2.csv', index_col='Accession')
lcms_3 = pd.read_csv(data_dir + 'lcsms_file_3.csv', index_col='Accession') 
lcms_4 = pd.read_csv(data_dir + 'lcsms_file_4.csv', index_col='Accession') 
lcms_5 = pd.read_csv(data_dir + 'lcsms_file_5.csv', index_col='Accession')

"""selecting LCMS files to use in below analysis"""
lcms_files = [lcms_1, lcms_2, lcms_3, lcms_4, lcms_5,]

"""Merging all files """
def merge_lcms(lcms_files):
    """get number of files"""
    x = len(lcms_files)
    merged_frame = pd.DataFrame(columns = ["Accession", "drop"])
    merged_frame.set_index("Accession", inplace=True)
    for i in lcms_files:
        temp_columns = [col for col in i.columns if "spec" in col]
        temp_frame = i[temp_columns]
        merged_frame = merged_frame.merge(temp_frame, left_index=True, right_index=True, how='outer')
    merged_frame.drop('drop', axis=1, inplace=True)
    return merged_frame
    
raw_data_df = merge_lcms(lcms_files)
raw_data_df

In [ ]:
"""Count Normalization"""
def normalize_lcms(raw_data_df):
    column_counts = {}
    norm_dict = {}
    """Get the column counts for each column"""
    for column in raw_data_df.columns.tolist():
        column_counts[column] = raw_data_df[column].sum()
    
    """Get the normalization value for each column"""
    for column in raw_data_df.columns.tolist():
        norm_dict[column] = Series([*column_counts.values()]).mean() / column_counts[column]
    
    norm_df = pd.DataFrame.from_dict(norm_dict, orient='index')
    """Apply normalization factors, remember norm_df is a dataframe and column 0 is where the values are stored"""
    normalized_data_df = raw_data_df.T.mul(norm_df[0], axis=0).T
    """Remove low counts"""
    normalized_data_df['max'] = normalized_data_df.max(axis=1)
    normalized_data_df = normalized_data_df[normalized_data_df['max'] > 2]
    normalized_data_df.drop('max', axis=1, inplace=True)
    normalized_data_df.fillna(0, inplace=True)
    return normalized_data_df

normed_df = normalize_lcms(raw_data)
normed_df.head(2)

In [ ]:
"""Quantile Normalization"""
def quantile_normalize(df):
    df_sorted = pd.DataFrame(np.sort(df.values,
                                     axis=0), 
                             index=df.index, 
                             columns=df.columns)
    df_mean = df_sorted.mean(axis=1)
    df_mean.index = np.arange(1, len(df_mean) + 1)
    df_qn =df.rank(method="min").stack().astype(int).map(df_mean).unstack()
    return(df_qn)
quant_normed = quantile_normalize(raw_data)
quant_normed.fillna(0, inplace=True)
quant_normed.head(2)
quant_normed.isnull().sum().sum()

In [ ]:
"""Removing short hypothetical proteins from the data, as they are likely part of larger unknown proteins that remain unidentified
Returns raw data for DESeq, count normalized and quantile normalized frames filtered for short proteins"""
def length_filter(raw_df, annotations_df, normalized_df, quant_df):
    """get lenght df"""
    lenght_df = annotations_df[['locus_tag', 'aa_seq']]
    lenght_df['length'] = lenght_df['aa_seq'].str.len()
    lenght_df.set_index(['locus_tag'], inplace=True)
    
    """get length filtered indicies"""
    short_indicies = set(set(lenght_df[lenght_df['length'] < 100].index.tolist()) & 
                        set(raw_df.index.tolist()))
    long_idicies = set(set(lenght_df[lenght_df['length'] > 100].index.tolist()) & 
                      set(raw_df.index.tolist()))
    
    """Filter for long indicies"""
    filtered_df = raw_df.loc[long_idicies].copy()
    filtered_df.fillna(0, inplace=True)
    
    """get normalized filtered"""
    norm_filtered = normalized_df.loc[set(set(normalized_df.index.tolist()) & set(filtered_df.index.tolist()))]
    quant_filtered = quant_df.loc[set(set(quant_df.index.tolist()) & set(filtered_df.index.tolist()))]
    return filtered_df, norm_filtered, quant_filtered

deseq_data, norm_filtered, quant_filtered = length_filter(subset_data, es894_annotations, normed_df, quant_normed)

"""Making transposed DF here for ease of use"""
transposed = norm_filtered.T.merge(metadata, left_index=True, right_index=True)
transposed_quant = quant_filtered.T.merge(metadata, left_index=True, right_index=True)


In [ ]:
"""Looking at coefficient of variation plots as quick check of quality"""
def coefficient_var_plot(raw_data, metadata):
    sns.set(style='darkgrid', context='talk')
    plotting_frame = raw_data.T.merge(metadata[['batch']], left_index=True, right_index=True, how='left')
    groups = plotting_frame['batch'].unique().tolist()
    print(groups)
    for i in groups:
        sample_plot = plotting_frame[plotting_frame['batch'] == i]
        sample_plot.drop(['batch'], axis=1, inplace=True)
        sample_plot = sample_plot.T
        
        sample_plot['mean'] = sample_plot.mean(axis=1)
#         print(sample_df.head())
        sample_plot['cv'] = stats.variation(sample_plot.drop('mean', axis=1), axis=1)
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.regplot(x=sample_plot['mean'], y=sample_plot['cv'], lowess=True, 
                    scatter_kws={"color": "black", "s":40, "linewidth":.5}, line_kws={"color": "blue", "linewidth":2},
                    marker='x' )
        ax.set_ylim(-0.1, 2)
        ax.set_xlim(-10, 400)
        ax.set_title(str(i + ' Coefficient of Variation'))
        ax.set_xlabel('Protein Spectral Counts')
        ax.set_ylabel('CV')
        
coefficient_var_plot(deseq_data, metadata)

DESeq

In [ ]:
"""DESeq2 FUNCTIONS"""
"""Necessary to run DESeq2 before running the variance stabilizing transformation"""
def run_deseq_in_python(counts_frame, sample_frame, exp_design):
    """Importing rpy2 functions"""
    from rpy2 import robjects
    import rpy2.robjects.numpy2ri
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri, Formula
    robjects.numpy2ri.activate()
    pandas2ri.activate()
    deseq = importr('DESeq2')
    
    """Creating R dfs"""
    count_matrix = robjects.DataFrame(counts_frame)
    sample_matrix = robjects.DataFrame(sample_frame)
    
    """Running DESeq"""
    design = Formula(exp_design)
    ddsFullCountTable = deseq.DESeqDataSetFromMatrix(countData = count_matrix, colData = sample_matrix, design = design)
    dds = deseq.DESeq(ddsFullCountTable)
    
    """Getting comparisons"""
    contrasts = deseq.resultsNames(dds).copy().tolist()
    contrasts.remove('Intercept')
    
    return(dds, contrasts)
   
def get_deseq_results(dds, contrasts):
    """Import and Helper functions"""
    from rpy2 import robjects
    from rpy2.robjects.packages import importr
    deseq = importr('DESeq2')
    to_dataframe = robjects.r('function(x) data.frame(x)')
    """Building dictionaries of dfs"""
    fold_change_dict = {}
    sig_fc_dict = {}
    for i in contrasts:
        res = deseq.results(dds, contrast = [i])
        fold_change_dict[i] = to_dataframe(res)
        sig_fc_dict[i] = fold_change_dict[i][fold_change_dict[i]['padj'] < 0.05].copy()
        
    return fold_change_dict, sig_fc_dict

"""Runs DESeq on whole frame"""
dds, contrasts = run_deseq_in_python(deseq_data, metadata, 
                                     "~ cond_1")


In [ ]:
"""RUNNING DESeq NORMALIZATION"""
"""In cell below, nothing can come before the %%R line, or be on the line with it other than the inputs.  Comments even on the 
line below seemed to be causing problems"""
%load_ext rpy2.ipython

In [ ]:
%%R -i dds -o v

library('DESeq2')
vsd = varianceStabilizingTransformation(dds)
v = assay(vsd)

In [ ]:
"""TRANSLATING BACK INTO PYTHON DATAFRAME"""
vst_frame = pd.DataFrame(v)
vst_frame.columns = deseq_data.columns
vst_frame.index = deseq_data.index


CLUSTERING VISUALIZATIONS

In [ ]:
"""PCA PLOTS"""
"""PCA with dataFrames from list.  Loadings can be returned for the last datframe in the list to get the most important
genes.  Not sure how usefull this may prove, so leaving it basic for now."""
def run_pca(data_frame_list, color):
    loading_dict = {}
    ix = 0
    for df in data_frame_list:
        """Getting the indexes for each pca frame"""
        indicies = df.columns.tolist()
        
        pca = PCA(n_components=5)
        pca.fit(df.T)
        variance = pca.explained_variance_ratio_
        pca_frame = pd.DataFrame(pca.fit_transform(df.T))
        pca_frame.index = indicies
        
        pca_frame = pca_frame.merge(metadata[['batch', 'type', 'cond_1', 'cond_2']], left_index=True, 
                                    right_index=True, how='left')
        
        pca_frame.columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5',
                             'batch', 'type', 'cond_1', 'cond_2']


        sns.set(style='darkgrid', context='talk')
        fig, ax = plt.subplots(figsize=(20, 15))
        g = sns.scatterplot(x=pca_frame['PC1'], y=pca_frame['PC2'], hue=pca_frame[color], s=200)
        g.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1)
        g.set_title("PCA")
        plt.tight_layout()
        plt.show()
        print(variance)
        loadings = pd.DataFrame(np.abs(pca.components_.T))
        loadings.index = df.index.copy()
        loading_dict[ix] = loadings
        ix += 1
#         return loading_dict
    
"""For all samples"""
run_pca([deseq_data, norm_filtered, quant_normed], 'batch')    




In [ ]:
"""MDS PLOTS"""
def run_mds(dataframe_list, color, metadata):
    for df in dataframe_list:
        """Getting indicies"""
        indicies = df.columns.tolist()
        
        all_bc_dm = beta_diversity('braycurtis', (df.T + 1).astype('int64').values)
        pco_all = pcoa(all_bc_dm)
        pco_df = pco_all.samples
        var_exp = pco_all.proportion_explained
        plot_data = pco_all.samples.iloc[:, [0,1]]
        plot_data = plot_data.div(plot_data.abs().max(axis=0), axis=1)
        
        """Merging in transposed df metadata"""
        plot_data.index = indicies
        plot_data = plot_data.merge(metadata[['batch', 'type', 'cond_1', 'cond_2', ]], 
                                    left_index=True, right_index=True, how='left')
        
        
        sns.set(style='darkgrid', context='talk')
        fig, ax = plt.subplots(figsize=(20, 15))
        g = sns.scatterplot(x=plot_data['PC1'], y=plot_data['PC2'], hue=plot_data[color], s=200)
        g.set_xlabel(str('PC1 ' + str(np.round(var_exp[0] * 100, 1)) + '%'))
        g.set_ylabel(str('PC2 ' + str(np.round(var_exp[1] * 100, 1)) + '%'))
        g.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1)
        g.set_title("PCO")
        plt.tight_layout()

"""Run for all samples"""
run_mds([deseq_data, norm_filtered, quant_filtered, ], 'batch', metadata)

In [ ]:
"""TSNE PLOTS"""
def run_tsne(dataframe_list, color):
    for df in dataframe_list:
        tsne = TSNE(n_components=2, verbose=1, perplexity=5, n_iter=5000)
        tsne_results = tsne.fit_transform(df.T)
        tsne_plotting = pd.DataFrame(tsne_results, columns=['Component_1', 'Component_2'])
        tsne_plotting.index = df.T.index
        tsne_plotting = tsne_plotting.merge(metadata[['batch', 'type', 'cond_1', 'cond_2']], left_index=True, 
                                            right_index=True, how='left')
        
        sns.set(style='darkgrid', context='talk')
        fig, ax = plt.subplots(figsize=(20, 15))
        g = sns.scatterplot(x=tsne_plotting['Component_1'], y=tsne_plotting['Component_2'], hue=tsne_plotting[color], s=200)
        g.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1)
        g.set_title("TSNE")
        #     g.set(ylim=(-1.2, 1.2), xlim=(-1.2, 1.2))
        plt.tight_layout()
        
"""Run for all samples"""
run_tsne([deseq_data, norm_filtered, vst_frame], 'batch')


In [ ]:
"""HEATMAPS"""
def run_heirarchical(dataframe_list):
    from matplotlib.colors import LogNorm
    import math
    for df in dataframe_list:
        cmap = sns.cm.rocket_r
#         cmap = sns.diverging_palette(200, 20, as_cmap=True)
        heat_map_data = df.copy()
        heat_map_data[heat_map_data < 1] = 1
        log_norm = LogNorm(vmin=heat_map_data.min().min(), vmax=heat_map_data.max().max())
        cbar_ticks = [math.pow(10, i) for i in range(math.floor(math.log10(heat_map_data.min().min())), 
                                                     1+math.ceil(math.log10(heat_map_data.max().max())))]
        """Either one of these works, so maybe can introduce another column to merge in the gene names"""
        g = sns.clustermap(heat_map_data, metric='braycurtis', norm=log_norm, cmap="RdBu_r", cbar_kws={"ticks": cbar_ticks}, 
                           xticklabels=True, yticklabels=0, figsize=(14,10))
        # g = sns.clustermap(heat_map_data,
        #                    norm=log_norm, cmap=cmap, vmin=1E6, cbar_kws={"ticks": cbar_ticks}, yticklabels=1, figsize=(14,10))

        g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize=0)
        g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels(), fontsize=10)
        g.ax_row_dendrogram.set_visible(False)
        g.cax.set_position([.10, .59, .03, .25])
    
"""Run for all samples"""    
run_heirarchical([deseq_data, norm_filtered, quant_filtered])


In [ ]:
"""HEATMAPS ZSCORE"""
def run_heirarchical(dataframe_list):
    from matplotlib.colors import LogNorm
    import math
    for df in dataframe_list:
        cmap = sns.cm.rocket_r
        heat_map_data = df.copy()
        heat_map_data[heat_map_data < 1] = 1
        """Either one of these works, so maybe can introduce another column to merge in the gene names"""
        g = sns.clustermap(heat_map_data, z_score=0, cmap="RdBu_r", 
                           xticklabels=True, yticklabels=0,  col_cluster=True, row_cluster=True, figsize=(14,10))
        # g = sns.clustermap(heat_map_data,
        #                    norm=log_norm, cmap=cmap, vmin=1E6, cbar_kws={"ticks": cbar_ticks}, yticklabels=1, figsize=(14,10))

        g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize=0)
        g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels(), fontsize=10)
        g.ax_row_dendrogram.set_visible(False)
        g.cax.set_position([.10, .59, .03, .25])
    
"""Run for all samples"""    
run_heirarchical([norm_filtered])

DIFFERENTIAL EXPRESSION TESTING

In [ ]:
"""Runs DESeq of all groups vs all groups comparisions, yielding dictionary of dataframes containing DESeq results for each possible comparison
within the set of comparators """

def get_deseq_comparison(dds, contrast_list, variable, transposed_df):
    """Import and Helper functions"""
    from rpy2 import robjects
    from rpy2.robjects.packages import importr
    deseq = importr('DESeq2')
    to_dataframe = robjects.r('function(x) data.frame(x)')
    
    """Does all vs all comparisons through the list iteration method
    Introducing this so that the foldchange frames can be more easily matched up with the 
    appropriate sample data"""
    fold_change_dict = {}
    for i in range(len(contrast_list)):
        j = i
        while j + 1 < len(contrast_list):
            """Getting DESeq results"""
            contrast_name = str(contrast_list[i] + '_v_' + contrast_list[j+1])
            contrast_array = np.asarray([variable, contrast_list[i], contrast_list[j+1]])
            res = deseq.results(dds, contrast = contrast_array)
            
            """Merging with mean values for appropriate comparisons"""
            pre_pivot = transposed_df[transposed_df['process'].isin([contrast_list[i], contrast_list[j+1]])].copy()
            pre_pivot.drop(['batch', 'cond_1', 'cond_2', 'cond_3', 'type'], 
                           axis=1, inplace=True)
            pivoted = pre_pivot.pivot_table(index='process', aggfunc='mean').T
            pivoted['read_max'] = pivoted.max(axis=1)
            """Turn R df into pandas and merge with values from pivoted"""
            fold_change_dict[contrast_name] = to_dataframe(res).merge(pivoted, left_index=True, right_index=True,
                                                                     how='left')

            
            j+=1
    return fold_change_dict, pivoted
fold_change_dict, pivoted = get_deseq_comparison(dds, ['cond_1', 'cond_2', 'cond_3'], 'process', transposed)
 """For saving fc dict to file"""
for key in fold_change_dict:
    fold_change_dict[key].to_csv(data_dir + '/process_' + key + '_fc.csv')
   

In [ ]:
"""MAKING FOLD CHANGE PLOTS FOR ALL USER DEFINED COMPARISONS"""
def volcano_plots(fold_change_dict):
    keys = fold_change_dict.keys()
    for key in keys:
        plotting_frame = fold_change_dict[key].copy()
        plotting_frame = plotting_frame[plotting_frame['read_max'] > 5]
#         print(plotting_frame)
        plotting_frame['sig'] = 0
#         plotting_frame['sig'] = np.where(plotting_frame['padj'] > 0.05, 0, 1)
        plotting_frame['sig'] = np.where((np.abs(plotting_frame['log2FoldChange']) > 1) & (plotting_frame['padj'] < 0.05), 
                                         1, 0)
#         print(plotting_frame)
        sns.set(style='darkgrid', context='talk')
        fig, ax = plt.subplots(figsize=(10, 6))
        g = sns.scatterplot(x=plotting_frame['log2FoldChange'], y=-np.log10(plotting_frame['padj']), 
                            hue=plotting_frame['sig'],legend=False, s=50, style=plotting_frame['sig'], 
                            markers=["o", "o"])
#         palette=['black','red']
#         g.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1)
        g.set_ylabel('-log10 P-Value')
        g.set_xlabel('log2 Fold Change')
        g.set_title(key)
        g.set(ylim=(-0.5, 20), xlim=(-7, 7))
        plt.tight_layout()
        plt.savefig(data_dir + '/process_' + key + '_volcano_min5.png')


volcano_plots(fold_change_dict)

In [ ]:
"""Annotate Differentially expressed proteins"""
def annotated_diff(fold_change_dict, annotations, read_thresh):
    keys = fold_change_dict.keys()
    sig_annot_dict = {}
    for key in keys:
        plotting_frame = fold_change_dict[key].copy()
#         print(plotting_frame)
        plotting_frame['sig'] = 0
#         plotting_frame['sig'] = np.where(plotting_frame['padj'] > 0.05, 0, 1)
        plotting_frame['sig'] = np.where((np.abs(plotting_frame['log2FoldChange']) > 1) & (plotting_frame['padj'] < 0.05) & 
                                         (plotting_frame['read_max'] > read_thresh), 1, 0)
        diff_frame = plotting_frame.copy()
        
        annotated_diff = diff_frame.merge(annotations[['locus_tag', 'name', 'uniprot_kwds', 'go_kwds']], 
                                       left_index=True, right_on='locus_tag', how='left')
        annotated_diff.sort_values(by='read_max', ascending=False, inplace=True)
        annotated_diff = annotated_diff[annotated_diff['sig'] == 1]
        annotated_diff.drop_duplicates(subset='locus_tag', keep='first', inplace=True)
        sig_annot_dict[key] = annotated_diff.drop(['baseMean', 'lfcSE', 'stat', 'pvalue',
                                                  'sig'], axis=1)
        
    return sig_annot_dict

sig_annotated_dict = annotated_diff(fold_change_dict, genome_annotations, 5)
